In [11]:
import cv2
import os
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import scale
from math import ceil
import matplotlib.pyplot as plt
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import RMSprop, Adam

In [12]:
path_test='../data/kaggle_dogcat/test'
path_train_cats='../data/kaggle_dogcat/train/cats'
path_train_dogs='../data/kaggle_dogcat/train/dogs'
def getimage(path):
    images=[]
    for i in os.listdir(path):
        img=cv2.resize(cv2.imread(os.path.join(path,i)), (32,32), interpolation=cv2.INTER_LINEAR )
        images.append(img)
    return images

In [13]:
train_cats=np.asarray(getimage(path_train_cats))
train_dogs=np.asarray(getimage(path_train_dogs))
train=np.append(train_dogs,train_cats,axis=0)

In [14]:
zero=np.zeros(len(train_cats))
one=np.ones(len(train_dogs))
test=np.append(zero,one)
test=test.reshape(4000,1)

In [15]:
x_train, x_test,y_train, y_test = train_test_split(train,test,test_size=0.33, random_state=42)

In [16]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train / 255.
x_test = x_test / 255.
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [17]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_12 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_13 (Activation)   (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 15, 15, 64)        18496     
__________

In [18]:
batch_size = 32 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 30 # 訓練整個資料集共 30個循環

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 2680 samples, validate on 1320 samples
Epoch 1/30
2680/2680 [==============================] - 1s 451us/sample - loss: 0.7658 - acc: 0.4866 - val_loss: 0.6989 - val_acc: 0.5212
Epoch 2/30
2680/2680 [==============================] - 1s 229us/sample - loss: 0.7028 - acc: 0.5246 - val_loss: 0.6895 - val_acc: 0.5841
Epoch 3/30
2680/2680 [==============================] - 1s 231us/sample - loss: 0.6777 - acc: 0.5802 - val_loss: 0.6875 - val_acc: 0.5303
Epoch 4/30
2680/2680 [==============================] - 1s 233us/sample - loss: 0.6556 - acc: 0.6160 - val_loss: 0.6352 - val_acc: 0.6417
Epoch 5/30
2680/2680 [==============================] - 1s 231us/sample - loss: 0.6226 - acc: 0.6634 - val_loss: 0.6076 - val_acc: 0.6705
Epoch 6/30
2680/2680 [==============================] - 1s 230us/sample - loss: 0.6097 - acc: 0.6672 - val_loss: 0.6214 - val_acc: 0.6758
Epoch 7/30
2680/2680 [==============================] - 1s 234us/sample - loss: 0.5791 - acc: 0.6955 - val_loss: 0.6027 - va

In [43]:
test_img=np.asarray(getimage(path_test))

In [44]:
prediction=model.predict_classes(test_img)

In [45]:
ID=np.arange(0,400,1)

In [50]:
import pandas as pd
prediction=pd.DataFrame(prediction)
ID=pd.DataFrame(ID)

In [61]:
prediction.columns=["prediction"]
ID.columns=["ID"]

In [75]:
out=pd.concat([ID,prediction],axis=1)

In [77]:
out.to_csv("final.csv", header=["ID", "Predicted"], index=False)